In [1]:
from operations import *
from reductions import *
from link_compile import CpuReduc
import time

In [6]:
start = time.time()
red_formula_string = "Sum_Reduction((Exp(Minus(Sum(Square((Var(0,3,0) - Var(1,3,1))*Var(1,3,1))))) * Var(2,1,1)),0)"
red_formula = eval(red_formula_string)
code = CpuReduc(red_formula,dtype="float",dtypeacc="double")()
f = open("test.cpp","w")
f.write(code)
f.close()
elapsed = time.time()-start
print("code generation time : ", elapsed)

start = time.time()
!g++ -c -O3 -DUSE_OPENMP=1 test.cpp
elapsed = time.time()-start
print("compilation time : ", elapsed)

code generation time :  0.002766132354736328
compilation time :  0.24523091316223145


In [3]:
!clang-format -i test.cpp
f = open("test.cpp")
code = f.read()
f.close()
print(code)

#include <cmath>

#ifdef USE_OPENMP
#include <omp.h>
#endif

extern "C" int CpuReduc(int nx, int ny, float *out, float **args) {

#pragma omp parallel for
  for (int i = 0; i < nx; i++) {
    float fout[1];
    float xi[3];
    float yj[4];
    double acc[1];
    xi[0] = args[0][i * 3 + 0];
    xi[1] = args[0][i * 3 + 1];
    xi[2] = args[0][i * 3 + 2];

#pragma unroll
    for (int k = 0; k < 1; k++)
      acc[k] = (double)(0.0f);
    for (int j = 0; j < ny; j++) {
      yj[0] = args[2][j * 1 + 0];
      yj[1] = args[1][j * 3 + 0];
      yj[2] = args[1][j * 3 + 1];
      yj[3] = args[1][j * 3 + 2];

      // Starting code block for Mult.
      {
        float arg0_1[1];

        // Starting code block for Exp.
        {
          float arg_1[1];

          // Starting code block for Minus.
          {
            float arg_2[1];

            // Starting code block for Sum.
            {
              float arg_3[3];

              // Starting code block for Square.
              {
    